# Imports and Definitions

In [13]:
import pandas as pd
from glob import glob
from tqdm import tqdm
import os
import random

In [2]:
root = "/scratch/tsoares/wikidumps/simplewiki-NS0-20231001/processed_data/"

# Load data

In [3]:
link_files = glob(os.path.join(root, "good_links*"))
page_files = glob(os.path.join(root, "good_pages*"))
link_files.sort()
page_files.sort()

In [4]:
print(link_files)
print(page_files)

['/scratch/tsoares/wikidumps/simplewiki-NS0-20231001/processed_data/good_links_0.parquet', '/scratch/tsoares/wikidumps/simplewiki-NS0-20231001/processed_data/good_links_1.parquet']
['/scratch/tsoares/wikidumps/simplewiki-NS0-20231001/processed_data/good_pages_0.parquet', '/scratch/tsoares/wikidumps/simplewiki-NS0-20231001/processed_data/good_pages_1.parquet']


In [5]:
dfs = []
for file in link_files:
    dfs.append(pd.read_parquet(file))
df_links = pd.concat(dfs)
df_links = df_links.sample(100_000).reset_index(drop=True)
df_links

,target_title,target_section,source_title,target_ID,target_QID,source_ID,source_QID,mention,source_section,link_start_index,link_end_index,sentence,sentence_raw,sentence_start_index,sentence_end_index,source_page_length,link_section_depth,context
0,Beth_Carvalho,Lead,May_5,692749,Q458663,9896,Q2550,Beth Carvalho,Births,107610,107704,"1946 - Beth Carvalho, Brazilian samba singer (...","<li id=""mwAr0""><a rel=""mw:WikiLink"" href=""./19...",107525,107807,163355,2.0.0,"politician (d. 2011) 1931 - Greg, Belgian cart..."
1,Anglicanism,Lead,St._Luke%27s_Church%2C_Abbottabad,61583,Q6423963,367920,Q1659212,Anglican,Lead,1383,1481,"Luke's Church, Abbottabad is an Anglican chur...","Luke's Church, Abbottabad</b> is an <a rel=""m...",1346,1875,6695,0.0.0,St. It was originally founded in the town of A...
2,Latin,Lead,Pope_Benedict_XVI,6592,Q397,9330,Q2494,Latin,Lead,15530,15817,Pope Benedict XVI (Latin: Benedictus PP.,"<p id=""mwBg""><b id=""mwBw"">Pope Benedict XVI</b...",15481,15889,97043,0.0.0,XVI; German: Benedikt XVI; Italian: Benedetto ...
3,Emigration,Lead,Virginia_Dighero,112427,Q187668,632076,Q1403436,emigrant,Lead,2593,2691,"[2] On 14 September 2005, she became the longe...","<sup about=""#mwt10"" class=""mw-ref reference"" i...",2094,2793,15054,0.0.0,"Virginia Dighero (married Zolezzi, 24 December..."
4,Arithmetic_logic_unit,Lead,Electronics,139860,Q192903,3659,Q11650,Arithmetic logic unit,Digital circuits,14114,14231,Arithmetic logic unit\n,"<li id=""mwZA""><a rel=""mw:WikiLink"" href=""./Ari...",14100,14237,30840,4.0.0,Digital circuits Digital circuits are used for...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,Ohio,Lead,Clinton_County%2C_Ohio,9240,Q1397,759946,Q113875,Ohio,Lead,2596,2662,Clinton County is a county in the U.S. state o...,"<p id=""mwBg""><b id=""mwBw"">Clinton County</b> i...",2346,2663,46235,0.0.0,"In 2020, 42,018 people lived there. The county..."
99996,1486,Lead,November_13,39590,Q6646,8712,Q2984,1486,Births,66456,66523,"1486 - Johann Eck, German theologian (d. 1543)\n","<li id=""mwATU""><a rel=""mw:WikiLink"" href=""./14...",66441,66959,157654,2.0.0,"Births 354 – Augustine of Hippo, Algerian phil..."
99997,Actor,Lead,November_29,6371,Q33999,10091,Q3016,actress,Births,120367,120463,"1982 - Gemma Chan, English actress\n","<li id=""mwA1c""><a rel=""mw:WikiLink"" href=""./19...",120187,120469,164901,2.0.0,"footballer 1968 - Hayabusa, Japanese wrestler ..."
99998,Plymouth_County%2C_Massachusetts,Lead,Whitman%2C_Massachusetts,729262,Q54086,980688,Q2536900,Plymouth County,Lead,1397,1526,"Whitman is a town in Plymouth County, Massachu...","<section data-mw-section-id=""0"" id=""mwAQ""><p i...",16,1717,27508,0.0.0,"v t e Plymouth County, Massachusetts United St..."


In [6]:
dfs = []
for file in page_files:
    dfs.append(pd.read_parquet(file, columns=['title', 'lead_paragraph']))
df_pages = pd.concat(dfs)
df_pages

,title,lead_paragraph
1,1283,1283 was a common year.
4,1296,The year 1296 was a leap year which started on...
6,12_Rounds,12 Rounds is a British rock band that was star...
8,127_Hours,127 Hours is a 2010 American biographical dram...
9,12_Monkeys,12 Monkeys is a 1995 American science fiction ...
...,...,...
39513,Aure,Aure can be
39516,Gol%2C_Norway,"Gol is a municipality in Viken county, Norway...."
39517,Sel%2C_Norway,"Sel is a municipality in Innlandet county, Nor..."
39518,Stranda,Stranda is a municipality in Møre og Romsdal c...


# Create auxiliary data structures

In [53]:
source_to_all_targets = {}
target_to_all_sources = {}
for i in tqdm(range(len(df_links))):
    source = df_links.iloc[i]['source_title']
    target = df_links.iloc[i]['target_title']
    source_section = df_links.iloc[i]['source_section'].split('<sep>')[0]
    if source not in source_to_all_targets:
        source_to_all_targets[source] = []
    source_to_all_targets[source].append({'target': target, 'section': source_section})
    if target not in target_to_all_sources:
        target_to_all_sources[target] = []
    target_to_all_sources[target].append(source)

  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [00:21<00:00, 4588.64it/s]


In [8]:
page_leads = {}
for i in tqdm(range(len(df_pages))):
    title = df_pages.iloc[i]['title']
    lead = df_pages.iloc[i]['lead_paragraph']
    page_leads[title] = lead

100%|██████████| 238680/238680 [00:14<00:00, 16554.44it/s]


# Set-up positive samples

In [61]:
positive_samples = []
removed = {'vte': 0, 'short': 0, 'non-alphanumeric': 0, 'short source': 0, 'short target': 0, 'target = source': 0}
for i in tqdm(range(len(df_links))):
    sample = {}
    sample['source_title'] = df_links.iloc[i]['source_title']
    sample['source_lead'] = page_leads[sample['source_title']]
    sample['target_title'] = df_links.iloc[i]['target_title']
    sample['target_lead'] = page_leads[sample['target_title']]
    sample['link_context'] = df_links.iloc[i]['context']
    sample['source_section'] = df_links.iloc[i]['source_section'].split('<sep>')[0]
    sample['label'] = 1

    if sample['source_title'] == sample['target_title']:
        removed['target = source'] += 1
        continue
    if len(sample['source_lead'].split(' ', 5)) < 6:
        removed['short source'] += 1
        continue
    if len(sample['target_lead'].split(' ', 5)) < 6:
        removed['short target'] += 1
        continue
    if len(sample['link_context'].split(' ', 5)) < 6:
        removed['short'] += 1
        continue
    if 'v t e' in sample['link_context']:
        removed['vte'] += 1
        continue
    if sum(c.isalnum() for c in sample['link_context']) < 0.5 * len(sample['link_context']):
        removed['non-alphanumeric'] += 1
        continue
    
    positive_samples.append(sample)

  0%|          | 0/100000 [00:00<?, ?it/s]

100%|██████████| 100000/100000 [00:35<00:00, 2791.57it/s]


In [33]:
print(removed)

{'vte': 2467, 'short': 2533, 'non-alphanumeric': 53, 'short source': 298, 'short target': 766, 'target = source': 181}


# Set-up negative samples

## Define hyper-parameters

In [37]:
negative_strategies = {
    'easy_replace_source': True,
    'easy_replace_target': True,
    'hard_replace_source': True,
    'hard_replace_target': True,
    'replace_context': True,
    'replace_section_source': True,
}
negative_samples_per_positive = 10

## Build negative samples from positive ones

In [38]:
strategies = [key for key in negative_strategies if negative_strategies[key]]
strategies

['easy_replace_source',
 'easy_replace_target',
 'hard_replace_source',
 'hard_replace_target',
 'replace_context',
 'replace_section_source']

In [66]:
negative_samples = []
for i in tqdm(range(len(positive_samples))):
    valid_strategies = strategies.copy()
    if len(source_to_all_targets[positive_samples[i]['source_title']]) == 1:
        valid_strategies.remove('hard_replace_target')
    if len(target_to_all_sources[positive_samples[i]['target_title']]) == 1:
        valid_strategies.remove('hard_replace_source')
    list_strategies = random.choices(valid_strategies, k=negative_samples_per_positive)
    new_samples = []
    for strategy in list_strategies:
        if strategy == 'easy_replace_source':
            new_source = random.choices(positive_samples, k=1)[0]['source_title']
            while new_source in target_to_all_sources[positive_samples[i]['target_title']]:
                new_source = random.choices(positive_samples, k=1)[0]['source_title']
            new_sample = positive_samples[i].copy()
            new_sample['source_title'] = new_source
            new_sample['source_lead'] = page_leads[new_source]
        elif strategy == 'easy_replace_target':
            new_target = random.choices(positive_samples, k=1)[0]['target_title']
            while new_target in source_to_all_targets[positive_samples[i]['source_title']]:
                new_target = random.choices(positive_samples, k=1)[0]['target_title']
            new_sample = positive_samples[i].copy()
            new_sample['target_title'] = new_target
            new_sample['target_lead'] = page_leads[new_target]
        elif strategy == 'hard_replace_source':
            new_source_section = random.choices(target_to_all_sources[positive_samples[i]['target_title']], k=1)[0]
            new_sample = positive_samples[i].copy()
            new_sample['source_title'] = new_source_section
            new_sample['source_lead'] = page_leads[new_source_section]
        elif strategy == 'hard_replace_target':
            safe_targets = [target['target'] for target in source_to_all_targets[positive_samples[i]['source_title']] if target['section'] != positive_samples[i]['source_section']]
            if len(safe_targets) == 0:
                new_target = random.choices(positive_samples, k=1)[0]['target_title']
                while new_target in source_to_all_targets[positive_samples[i]['source_title']]:
                    new_target = new_target = random.choices(positive_samples, k=1)[0]['target_title']
            else:
                new_target = random.choices(safe_targets, k=1)[0]
            new_sample = positive_samples[i].copy()
            new_sample['target_title'] = new_target
            new_sample['target_lead'] = page_leads[new_target]
        elif strategy == 'replace_context':
            new_sample = positive_samples[i].copy()
            new_context = random.choices(positive_samples, k=1)[0]['link_context']
            while new_context == positive_samples[i]['link_context']:
                new_context = random.choices(positive_samples, k=1)[0]['link_context']
            new_sample['link_context'] = new_context
        elif strategy == 'replace_section_source':
            new_sample = positive_samples[i].copy()
            new_source = random.choices(positive_samples, k=1)[0]['source_title']
            while new_source_section == positive_samples[i]['source_section']:
                new_source_section = random.choices(positive_samples, k=1)[0]['source_section']
            new_sample['source_section'] = new_source_section
        new_sample['label'] = 0
        new_samples.append(new_sample)
    negative_samples.extend(new_samples)    

  0%|          | 0/93702 [00:00<?, ?it/s]

100%|██████████| 93702/93702 [00:03<00:00, 23799.53it/s]
